# Dataset

In [ ]:
import torch.utils.data as data
from PIL import Image, ImageFile
import os

ImageFile.LOAD_TRUNCATED_IAMGES = True


# https://github.com/pytorch/vision/issues/81

def PIL_loader(path):
    try:
        with open(path, 'rb') as f:
            return Image.open(f).convert('RGB')
    except IOError:
        print('Cannot load image ' + path)


def default_reader(fileList):
    imgList = []
    with open(fileList, 'r') as file:
        for line in file.readlines():
            list_new = line.strip().split('\t')
            imgPath, label = list_new[0], list_new[1]
            imgList.append((imgPath, int(label)))
    return imgList


class ImageList(data.Dataset):
    def __init__(self, root, fileList, transform=None, list_reader=default_reader, loader=PIL_loader):
        self.root = root
        self.imgList = list_reader(fileList)
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        imgPath, target = self.imgList[index]
        img = self.loader(os.path.join(self.root, imgPath))

        if self.transform is not None:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self.imgList)

Dataset pair

In [ ]:
import torch.utils.data as data
from PIL import Image, ImageFile
import os

ImageFile.LOAD_TRUNCATED_IAMGES = True
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()


# https://github.com/pytorch/vision/issues/81

def PIL_loader_pair(path):
    try:
        with open(path, 'rb') as f:
            return Image.open(f).convert('RGB')
    except IOError:
        print('Cannot load image ' + path)


def default_reader_pair(fileList):
    imgList = []
    labelList = []
    with open(fileList, 'r') as file:
        for line in file.readlines():
            list_new = line.strip().split('\t')
            imgPath, imgOccPath, label = list_new[0], list_new[0], list_new[0]
            label = label[:-14]

            imgPath = label + '_' + '0001.jpg'
            imgOccPath = label + '_' + '0001.jpg'
            imgPair = imgPath + '\t' + imgOccPath
            imgList.append(imgPair)
            labelList.append(label)
        label_encoder.fit_transform(labelList)
        for i in range(len(labelList)):
          labelList[i] = label_encoder.transform([labelList[i]])
    return imgList, labelList


class ImageList_pair(data.Dataset):
    def __init__(self, root, fileList, transform=None, list_reader=default_reader_pair, loader=PIL_loader_pair):
        self.root = root
        self.imgList, self.labelList = list_reader(fileList)
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        imgPair, target = self.imgList[index], self.labelList[index]
        imgPath, imgOccPath = imgPair.split('\t')
        # if int(imgPath) < 10:
        #   img = self.loader(os.path.join(self.root, target, target + '_000' + imgPath + '.jpg'))
        # elif int(imgPath) >= 10 and int(imgPath) < 100: img = self.loader(os.path.join(self.root, target, target + '_00' + imgPath + '.jpg'))
        # else: img = self.loader(os.path.join(self.root, target, target + '_0' + imgPath + '.jpg'))

        # if int(imgOccPath) < 10:
        #   img_occ = self.loader(os.path.join(self.root, target, target + '_000' + imgOccPath + '.jpg'))
        # elif int(imgOccPath) >= 10 and int(imgOccPath) < 100: img_occ = self.loader(os.path.join(self.root, target, target + '_00' + imgOccPath + '.jpg'))
        # else: img_occ = self.loader(os.path.join(self.root, target, target + '_0' + imgOccPath + '.jpg'))
        img = self.loader(self.root + '/' + imgPath.split('_0')[0] + '/' + imgPath)
        img = torchvision.transforms.Resize((128, 128))(img)
        img_occ = self.loader(self.root + '/lfw_train/' + imgPath.split('_0')[0] + '/' + imgPath)

        if self.transform is not None:
            img = self.transform(img)
            img_occ = self.transform(img_occ)
        return img, img_occ, imgPair, target

    def __len__(self):
        return len(self.imgList)

Dataset pair mask

In [ ]:
import torch
import torch.utils.data as data
from PIL import Image, ImageFile
import os
import numpy as np
import scipy.io as sio

# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

ImageFile.LOAD_TRUNCATED_IAMGES = True

mask_file = np.loadtxt('C:/Users/Admin/Downloads/mean_mask.txt', dtype='float32')

# https://github.com/pytorch/vision/issues/81

def PIL_loader_pairmask(path):
    try:
        with open(path, 'rb') as f:
            return Image.open(f).convert('RGB')
    except IOError:
        print('Cannot load image ' + path)


def default_reader_pairmask(fileList):
    imgList = []
    with open(fileList, 'r') as file:
        for line in file.readlines():
            # imgPath, imgOccPath, MskPath, label = line.strip().split(' ')
            # imgPair = imgPath + '\t' + imgOccPath
            # imgList.append((imgPair, MskPath, int(label)))
            list_new = line.strip().split('\t')
            imgPath, imgOccPath, label = list_new[1], list_new[2], list_new[0]
            imgPair = imgPath + '\t' + imgOccPath
            imgList.append((imgPair, label))
    return imgList


class ImageList_pairmask(data.Dataset):
    def __init__(self, root, fileList, transform=None, list_reader=default_reader_pairmask, loader=PIL_loader_pairmask):
        self.root = root
        self.imgList = list_reader(fileList)
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        # imgPair, mskPath, target = self.imgList[index]
        # imgPath, imgOccPath = imgPair.split('\t')
        # img = self.loader(os.path.join(self.root, imgPath))
        # img_occ = self.loader(os.path.join(self.root, imgOccPath))

        imgPair, target = self.imgList[index]
        imgPath, imgOccPath = imgPair.split('\t')
        if int(imgPath) < 10:
          img = self.loader(os.path.join(self.root, target, target + '_000' + imgPath + '.jpg'))
        elif int(imgPath) >= 10 and int(imgPath) < 100: img = self.loader(os.path.join(self.root, target, target + '_00' + imgPath + '.jpg'))
        else: img = self.loader(os.path.join(self.root, target, target + '_0' + imgPath + '.jpg'))

        if int(imgOccPath) < 10:
          img_occ = self.loader(os.path.join(self.root, target, target + '_000' + imgOccPath + '.jpg'))
        elif int(imgOccPath) >= 10 and int(imgOccPath) < 100: img_occ = self.loader(os.path.join(self.root, target, target + '_00' + imgOccPath + '.jpg'))
        else: img_occ = self.loader(os.path.join(self.root, target, target + '_0' + imgOccPath + '.jpg'))

        # mask
        # load_data = sio.loadmat(os.path.join(self.root, mskPath))
        # mask = torch.from_numpy(load_data['final_mask']).float()
        mask = mask_file.reshape(512, 16, 16)

        if self.transform is not None:
            img = self.transform(img)
            img_occ = self.transform(img_occ)
        return img, img_occ, imgPair, mask, label_encoder.fit_transform([target])

    def __len__(self):
        return len(self.imgList)

VGG Face

In [ ]:
import os
import numpy as np
import PIL.Image
import torch
from torch.utils import data
import torchvision.transforms
import random


class VGG_Faces(data.Dataset):
    mean_bgr = np.array([93.5940, 104.7624, 129.1863])  # from resnet50_ft.prototxt

    def __init__(self, root, image_list_file, split='train_occ_7',
                 o_width=0, o_height=0, r_start=0, c_start=0, transform=True,
                 upper=None):
        """
        :param root: dataset directory
        :param image_list_file: contains image file names under root
        :param image_list_file: contains image file names under root
        :param id_label_dict: X[class_id] -> label
        :param split: train or valid
        :param transform:
        :param upper: max number of image used for debug
        """
        assert os.path.exists(root), "root: {} not found.".format(root)
        # 'E:/Coding/data/testdata'
        self.root = root
        assert os.path.exists(image_list_file), "image_list_file: {} not found.".format(image_list_file)
        self.image_list_file = image_list_file
        self.split = split
        self._transform = transform
        self.o_width = o_width
        self.o_height = o_height
        self.r_start = r_start
        self.c_start = c_start

        self.img_info = []
        with open(self.image_list_file, 'r') as f:
            for i, img_file in enumerate(f):
                img_file = img_file.strip()  # e.g. train/n004332/0317_01.jpg
                class_name = img_file.split("/")[1]  # like n004332
                self.img_info.append({
                    'cname': class_name,
                    'img': img_file
                })
                if i % 1000 == 0:
                    print("processing: {} images for {}".format(i, self.split))
                if upper and i == upper - 1:  # for debug purpose
                    break

        #np.random.seed(666)
        self.occ_idxs = np.random.randint(1, 21, size=len(self.img_info))
        print(len(self.occ_idxs))

    def __len__(self):
        return len(self.img_info)

    def __getitem__(self, index):
        info = self.img_info[index]
        occ_id = self.occ_idxs[index]
        img_file = info['img']  # e.g. train/A.J._Buckley/00000001.jpg
        img = PIL.Image.open(os.path.join(self.root, img_file))
        img = self.process_image_channels(img, os.path.join(self.root, img_file))

        img = np.array(img, dtype=np.uint8)
        assert len(img.shape) == 3  # assumes color images and no alpha channel
        # generate occ image
        img_occ = self.generate_occ_img(img, occ_id)
        self.save_occ_img(img_occ, occ_id, img_file)
        class_name = info['cname']
        if self._transform:
            return self.transform(img), self.transform(img_occ), img_file, class_name
        else:
            return img, img_occ, img_file, class_name

    def generate_occ_img(self, img, occ_id):
        occ_img_path = self.root + '/occ_pattern/occ' + str(occ_id) + '.jpg'
        occ_img = PIL.Image.open(occ_img_path)
        occ_img = torchvision.transforms.Resize((self.o_height, self.o_width))(occ_img)
        occ_img = np.array(occ_img, dtype=np.uint8)
        img_occ = np.copy(img)
        img_occ[self.r_start:self.r_start + self.o_height, self.c_start:self.c_start + self.o_width, :] = occ_img[:, :, :]
        return img_occ

    def transform(self, img):
        img = img[:, :, ::-1]  # RGB -> BGR
        img = img.astype(np.float32)
        img -= self.mean_bgr
        img = img.transpose(2, 0, 1)  # C x H x W
        img = torch.from_numpy(img).float()
        return img

    def save_occ_img(self, img_occ, occ_id, img_file):
        # train/A.J._Buckley/00000001.jpg
        class_name = img_file.split("/")[1]
        image_name = img_file.split("/")[2]
        image_id = image_name.split(".")[0]
        save_folder = self.root + '/' + self.split + '/' + class_name
        '''
        if False == os.path.exists(save_folder):

            os.makedirs(save_folder)
        '''
        save_path = save_folder + '/' + image_id + '_' + str(occ_id) + '.jpg'
        PIL.Image.fromarray(img_occ).save(save_path)

    def process_image_channels(self, img, image_path):
        # process the 4 channels .png
        if img.mode == 'RGBA':
            r, g, b, a = img.split()
            img = PIL.Image.merge("RGB", (r, g, b))
        # process the 1 channel image
        elif img.mode != 'RGB':
            img = img.convert("RGB")
            os.remove(image_path)
            img.save(image_path)
        return img

# Model

Net model

In [ ]:
import torch
import torch.nn as nn



# -------------------------------------- sphere network Begin --------------------------------------
class Block_net(nn.Module):
    def __init__(self, planes):
        super(Block_net, self).__init__()
        self.conv1 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.prelu1 = nn.PReLU(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.prelu2 = nn.PReLU(planes)

    def forward(self, x):
        return x + self.prelu2(self.conv2(self.prelu1(self.conv1(x))))


class sphere_net(nn.Module):
    def __init__(self, type=20, is_gray=False):
        super(sphere_net, self).__init__()
        block = Block_net
        if type is 20:
            layers = [1, 2, 4, 1]
        elif type is 64:
            layers = [3, 7, 16, 3]
        else:
            raise ValueError('sphere' + str(type) + " IS NOT SUPPORTED! (sphere20 or sphere64)")
        filter_list = [3, 64, 128, 256, 512]
        if is_gray:
            filter_list[0] = 1

        self.layer1 = self._make_layer(block, filter_list[0], filter_list[1], layers[0], stride=2)
        self.layer2 = self._make_layer(block, filter_list[1], filter_list[2], layers[1], stride=2)
        self.layer3 = self._make_layer(block, filter_list[2], filter_list[3], layers[2], stride=2)
        self.layer4 = self._make_layer(block, filter_list[3], filter_list[4], layers[3], stride=2)
        self.fc = nn.Linear(512 * 7 * 6, 512)

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                if m.bias is not None:
                    nn.init.xavier_uniform_(m.weight)
                    nn.init.constant_(m.bias, 0.0)
                else:
                    nn.init.normal_(m.weight, 0, 0.01)


    def _make_layer(self, block, inplanes, planes, blocks, stride):
        layers = []
        layers.append(nn.Conv2d(inplanes, planes, 3, stride, 1))
        layers.append(nn.PReLU(planes))
        for i in range(blocks):
            layers.append(block(planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    def save(self, file_path):
        with open(file_path, 'wb') as f:
            torch.save(self.state_dict(), f)


# -------------------------------------- sphere network END --------------------------------------

# ---------------------------------- LResNet50E-IR network Begin ----------------------------------

class BlockIR_net(nn.Module):
    def __init__(self, inplanes, planes, stride, dim_match):
        super(BlockIR_net, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.prelu1 = nn.PReLU(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)

        if dim_match:
            self.downsample = None
        else:
            self.downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu1(out)
        out = self.conv2(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out


class LResNet_net(nn.Module):

    def __init__(self, block, layers, filter_list, is_gray=False):
        self.inplanes = 64
        super(LResNet_net, self).__init__()
        # input is (mini-batch,3 or 1,112,96)
        # use (conv3x3, stride=1, padding=1) instead of (conv7x7, stride=2, padding=3)
        if is_gray:
            self.conv1 = nn.Conv2d(1, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)  # gray
        else:
            self.conv1 = nn.Conv2d(3, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(filter_list[0])
        self.prelu1 = nn.PReLU(filter_list[0])
        self.layer1 = self._make_layer(block, filter_list[0], filter_list[1], layers[0], stride=2)
        self.layer2 = self._make_layer(block, filter_list[1], filter_list[2], layers[1], stride=2)
        self.layer3 = self._make_layer(block, filter_list[2], filter_list[3], layers[2], stride=2)
        self.layer4 = self._make_layer(block, filter_list[3], filter_list[4], layers[3], stride=2)
        self.fc = nn.Sequential(
            nn.BatchNorm1d(filter_list[4] * 7 * 6),
            nn.Dropout(p=0.4),
            nn.Linear(filter_list[4] * 7 * 6, 512),
            nn.BatchNorm1d(512),  # fix gamma ???
        )

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)


    def _make_layer(self, block, inplanes, planes, blocks, stride):
        layers = []
        layers.append(block(inplanes, planes, stride, False))
        for i in range(1, blocks):
            layers.append(block(planes, planes, stride=1, dim_match=True))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.prelu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    def save(self, file_path):
        with open(file_path, 'wb') as f:
            torch.save(self.state_dict(), f)


def LResNet50E_IR_net(is_gray=False):
    filter_list = [64, 64, 128, 256, 512]
    layers = [3, 4, 14, 3]
    return LResNet_net(BlockIR_net, layers, filter_list, is_gray)
# ---------------------------------- LResNet50E-IR network End ----------------------------------

<>:23: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:23: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-c6231eb78abc>:23: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if type is 20:
<ipython-input-3-c6231eb78abc>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif type is 64:


Net mask model

In [ ]:
import torch
import torch.nn as nn

class BlockIR_netmask(nn.Module):
    def __init__(self, inplanes, planes, stride, dim_match):
        super(BlockIR_netmask, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.prelu1 = nn.PReLU(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)

        if dim_match:
            self.downsample = None
        else:
            self.downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu1(out)
        out = self.conv2(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out


class LResNet_netmask(nn.Module):

    def __init__(self, block, layers, filter_list, is_gray=False):
        self.inplanes = 64
        super(LResNet_netmask, self).__init__()
        # input is (mini-batch,3 or 1,112,96)
        # use (conv3x3, stride=1, padding=1) instead of (conv7x7, stride=2, padding=3)
        if is_gray:
            self.conv1 = nn.Conv2d(1, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)  # gray
        else:
            self.conv1 = nn.Conv2d(3, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(filter_list[0])
        self.prelu1 = nn.PReLU(filter_list[0])
        self.layer1 = self._make_layer(block, filter_list[0], filter_list[1], layers[0], stride=2)
        self.layer2 = self._make_layer(block, filter_list[1], filter_list[2], layers[1], stride=2)
        self.layer3 = self._make_layer(block, filter_list[2], filter_list[3], layers[2], stride=2)
        self.layer4 = self._make_layer(block, filter_list[3], filter_list[4], layers[3], stride=2)
        self.fc = nn.Sequential(
            # nn.BatchNorm1d(filter_list[4] * 256),
            nn.Dropout(p=0.4),
            nn.Linear(filter_list[4] * 256, 512),
            # nn.BatchNorm1d(512),  # fix gamma ???
        )

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)


    def _make_layer(self, block, inplanes, planes, blocks, stride):
        layers = []
        layers.append(block(inplanes, planes, stride, False))
        for i in range(1, blocks):
            layers.append(block(planes, planes, stride=1, dim_match=True))

        return nn.Sequential(*layers)

    def forward(self, x, mask):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.prelu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = x * mask

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def save(self, file_path):
        with open(file_path, 'wb') as f:
            torch.save(self.state_dict(), f)


def LResNet50E_IR_netmask(is_gray=False):
    filter_list = [64, 64, 128, 256, 512]
    layers = [3, 4, 14, 3]
    return LResNet_netmask(BlockIR_netmask, layers, filter_list, is_gray)
# ---------------------------------- LResNet50E-IR network End ----------------------------------

Net siamese model

In [ ]:
import torch
import torch.nn as nn

class BlockIR_netsia(nn.Module):
    def __init__(self, inplanes, planes, stride, dim_match):
        super(BlockIR_netsia, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.prelu1 = nn.PReLU(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)

        if dim_match:
            self.downsample = None
        else:
            self.downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu1(out)
        out = self.conv2(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out


# --------------------------------------  LResNet_IR_Sia network Begin --------------------------------------

class LResNet_Siamese(nn.Module):

    def __init__(self, block, layers, filter_list, is_gray=False):
        self.inplanes = 64
        super(LResNet_Siamese, self).__init__()
        # input is (mini-batch,3 or 1,112,96)
        # use (conv3x3, stride=1, padding=1) instead of (conv7x7, stride=2, padding=3)
        if is_gray:
            self.conv1 = nn.Conv2d(1, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)  # gray
        else:
            self.conv1 = nn.Conv2d(3, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(filter_list[0])
        self.prelu1 = nn.PReLU(filter_list[0])
        self.layer1 = self._make_layer(block, filter_list[0], filter_list[1], layers[0], stride=2)
        self.layer2 = self._make_layer(block, filter_list[1], filter_list[2], layers[1], stride=2)
        self.layer3 = self._make_layer(block, filter_list[2], filter_list[3], layers[2], stride=2)
        self.layer4 = self._make_layer(block, filter_list[3], filter_list[4], layers[3], stride=2)
        # --Begin--Siamese branch
        self.sia = nn.Sequential(
            # nn.BatchNorm2d(filter_list[4]),
            nn.Conv2d(filter_list[4], 512, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PReLU(filter_list[4]),
            nn.BatchNorm2d(filter_list[4]),
            nn.Sigmoid(),
        )
        # --End--Siamese branch
        self.fc = nn.Sequential(
            # nn.BatchNorm1d(filter_list[4] * 7 * 6),
            nn.BatchNorm1d(filter_list[4] * 8 * 8),
            nn.Dropout(p=0),  # No drop for sia dic
            # nn.Linear(filter_list[4] * 7 * 6, 512),
            nn.Linear(filter_list[4] * 8 * 8, 512),
            nn.BatchNorm1d(512),  # fix gamma ???
        )

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)


    def _make_layer(self, block, inplanes, planes, blocks, stride):
        layers = []
        layers.append(block(inplanes, planes, stride, False))
        for i in range(1, blocks):
            layers.append(block(planes, planes, stride=1, dim_match=True))

        return nn.Sequential(*layers)

    def forward(self, x1, x2):
        x = self.conv1(x1)
        x = self.bn1(x)
        x = self.prelu1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        f1 = self.layer4(x)

        x = self.conv1(x2)
        x = self.bn1(x)
        x = self.prelu1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        f2 = self.layer4(x)
        # --Begin--Sia branch
        f_diff = torch.add(f1, -1.0, f2)
        f_diff = torch.abs(f_diff)
        out = self.sia(f_diff)
        # --End-- Sia branch
        f1_masked = f1 * out
        f2_masked = f2 * out

        fc1 = f1_masked.view(f1_masked.size(0), -1)  # 256*(512*7*6)
        fc2 = f2_masked.view(f2_masked.size(0), -1)
        fc1 = self.fc(fc1)
        fc2 = self.fc(fc2)

        return f1_masked, f2_masked, fc1, fc2, f_diff, out

    def save(self, file_path):
        with open(file_path, 'wb') as f:
            torch.save(self.state_dict(), f)


def LResNet50E_IR_Sia(is_gray=False):
    filter_list = [64, 64, 128, 256, 512]
    layers = [3, 4, 14, 3]
    return LResNet_Siamese(BlockIR_netsia, layers, filter_list, is_gray)
# -------------------------------------- LResNet50E-IR_Sia network END --------------------------------------
# -------------------------------------- LResNet50E-IR network Begin ----------------------------------------
class LResNet_Sia(nn.Module):

    def __init__(self, block, layers, filter_list, is_gray=False):
        self.inplanes = 64
        super(LResNet_Sia, self).__init__()
        # input is (mini-batch,3 or 1,112,96)
        # use (conv3x3, stride=1, padding=1) instead of (conv7x7, stride=2, padding=3)
        if is_gray:
            self.conv1 = nn.Conv2d(1, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)  # gray
        else:
            self.conv1 = nn.Conv2d(3, filter_list[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(filter_list[0])
        self.prelu1 = nn.PReLU(filter_list[0])
        self.layer1 = self._make_layer(block, filter_list[0], filter_list[1], layers[0], stride=2)
        self.layer2 = self._make_layer(block, filter_list[1], filter_list[2], layers[1], stride=2)
        self.layer3 = self._make_layer(block, filter_list[2], filter_list[3], layers[2], stride=2)
        self.layer4 = self._make_layer(block, filter_list[3], filter_list[4], layers[3], stride=2)
        self.fc = nn.Sequential(
            nn.BatchNorm1d(filter_list[4] * 7 * 6),
            nn.Dropout(p=0.4),
            nn.Linear(filter_list[4] * 7 * 6, 512),
            nn.BatchNorm1d(512),  # fix gamma ???
        )

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)


    def _make_layer(self, block, inplanes, planes, blocks, stride):
        layers = []
        layers.append(block(inplanes, planes, stride, False))
        for i in range(1, blocks):
            layers.append(block(planes, planes, stride=1, dim_match=True))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.prelu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    def save(self, file_path):
        with open(file_path, 'wb') as f:
            torch.save(self.state_dict(), f)


def LResNet50E_IR(is_gray=False):
    filter_list = [64, 64, 128, 256, 512]
    layers = [3, 4, 14, 3]
    return LResNet_Sia(BlockIR_netsia, layers, filter_list, is_gray)
# ---------------------------------- LResNet50E-IR network End ----------------------------------

# Layer

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math

def cosine_sim(x1, x2, dim=1, eps=1e-8):
    ip = torch.mm(x1, x2.t())
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    return ip / torch.ger(w1,w2).clamp(min=eps)

class MarginCosineProduct(nn.Module):
    r"""Implement of large margin cosine distance: :
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        s: norm of input feature
        m: margin
    """

    def __init__(self, in_features, out_features, s=30.0, m=0.35):
        super(MarginCosineProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        #stdv = 1. / math.sqrt(self.weight.size(1))
        #self.weight.data.uniform_(-stdv, stdv)

    def forward(self, input, label):
        cosine = cosine_sim(input, self.weight)
        # cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        # --------------------------- convert label to one-hot ---------------------------
        # https://discuss.pytorch.org/t/convert-int-into-one-hot-format/507
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, torch.reshape(label, (1, -1)), 1.0)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = self.s * (cosine - one_hot * self.m)
        return output

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + 'in_features=' + str(self.in_features) \
               + ', out_features=' + str(self.out_features) \
               + ', s=' + str(self.s) \
               + ', m=' + str(self.m) + ')'


class AngleLinear(nn.Module):
    def __init__(self, in_features, out_features, m=4):
        super(AngleLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        self.base = 1000.0
        self.gamma = 0.12
        self.power = 1
        self.LambdaMin = 5.0
        self.iter = 0
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        # duplication formula
        self.mlambda = [
            lambda x: x ** 0,
            lambda x: x ** 1,
            lambda x: 2 * x ** 2 - 1,
            lambda x: 4 * x ** 3 - 3 * x,
            lambda x: 8 * x ** 4 - 8 * x ** 2 + 1,
            lambda x: 16 * x ** 5 - 20 * x ** 3 + 5 * x
        ]

    def forward(self, input, label):
        # lambda = max(lambda_min,base*(1+gamma*iteration)^(-power))
        self.iter += 1
        self.lamb = max(self.LambdaMin, self.base * (1 + self.gamma * self.iter) ** (-1 * self.power))

        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cos_theta = F.linear(F.normalize(input), F.normalize(self.weight))
        cos_theta = cos_theta.clamp(-1, 1)
        cos_m_theta = self.mlambda[self.m](cos_theta)
        theta = cos_theta.data.acos()
        k = (self.m * theta / 3.14159265).floor()
        phi_theta = ((-1.0) ** k) * cos_m_theta - 2 * k
        NormOfFeature = torch.norm(input, 2, 1)

        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros_like(cos_theta)
        one_hot.scatter_(1, torch.reshape(label, (-1,1)), 1)

        # --------------------------- Calculate output ---------------------------
        output = (one_hot * (phi_theta - cos_theta) / (1 + self.lamb)) + cos_theta
        output *= NormOfFeature.view(-1, 1)

        return output

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + 'in_features=' + str(self.in_features) \
               + ', out_features=' + str(self.out_features) \
               + ', m=' + str(self.m) + ')'

# LFW evaluation

LFW evaluation

In [ ]:
from PIL import Image
import numpy as np

from torchvision.transforms import functional as F
import torchvision.transforms as transforms
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

cudnn.benchmark = True


def extractDeepFeature(img, model, is_gray):
    if is_gray:
        transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5,), std=(0.5,))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])
    else:
        transform = transforms.Compose([
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])


    '''
    img, img_ = transform(img), transform(F.hflip(img))
    img, img_ = img.unsqueeze(0).to('cuda'), img_.unsqueeze(0).to('cuda')
    ft = torch.cat((model(img), model(img_)), 1)[0].to('cpu')
    '''

    img = transform(img)
    img = img.unsqueeze(0).to('cuda')
    ft = model(img)[0].to('cpu')

    return ft


def KFold(n=6000, n_folds=10):
    folds = []
    base = list(range(n))
    for i in range(n_folds):
        test = base[int(i * n / n_folds):int((i + 1) * n / n_folds)]
        train = list(set(base) - set(test))
        folds.append([train, test])
    return folds


def eval_acc(threshold, diff):
    y_true = []
    y_predict = []
    for d in diff:
        same = 1 if float(d[0]) > threshold else 0
        y_predict.append(same)
        y_true.append(int(d[1]))
    y_true = np.array(y_true)
    y_predict = np.array(y_predict)
    accuracy = 1.0 * np.count_nonzero(y_true == y_predict) / len(y_true)
    return accuracy


def find_best_threshold(thresholds, predicts):
    best_threshold = best_acc = 0
    for threshold in thresholds:
        accuracy = eval_acc(threshold, predicts)
        if accuracy >= best_acc:
            best_acc = accuracy
            best_threshold = threshold
    return best_threshold


def eval(model, model_path=None, is_gray=False):
    predicts = []
    # checkpoint = torch.load(model_path)
    # model.load_state_dict(checkpoint['model_state_dict'])
    # model.eval()
    root = '/content/drive/MyDrive/dataset/lfw-deepfunneled/lfw-deepfunneled/'
    with open('/content/drive/MyDrive/dataset/pairs.txt') as f:
        pairs_lines = f.readlines()[1:]

    with torch.no_grad():
        for i in range(6000):
            p = pairs_lines[i].replace('\n', '').split('\t')

            if 3 == len(p):
                sameflag = 1
                name1 = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[1]))
                name2 = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[2]))
            elif 4 == len(p):
                sameflag = 0
                name1 = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[1]))
                name2 = p[2] + '/' + p[2] + '_' + '{:04}.jpg'.format(int(p[3]))
            else:
                raise ValueError("WRONG LINE IN 'pairs.txt! ")

            with open(root + name1, 'rb') as f:
                img1 =  Image.open(f).convert('RGB')
            with open(root + name2, 'rb') as f:
                img2 =  Image.open(f).convert('RGB')
            f1 = extractDeepFeature(img1, model, is_gray)
            f2 = extractDeepFeature(img2, model, is_gray)

            distance = f1.dot(f2) / (f1.norm() * f2.norm() + 1e-5)
            #predicts.append('{}\t{}\t{}\t{}\n'.format(name1, name2, distance, sameflag))
            predicts.append([distance, sameflag])

    accuracy = []
    thd = []
    folds = KFold(n=6000, n_folds=10)
    thresholds = np.arange(-1.0, 1.0, 0.005)
    #predicts = np.array(map(lambda line: line.strip('\n').split(), predicts))
    predicts = np.array(predicts)
    #print(type(predicts),predicts.shape,predicts[0])

    for idx, (train, test) in enumerate(folds):
        best_thresh = find_best_threshold(thresholds, predicts[train])
        accuracy.append(eval_acc(best_thresh, predicts[test]))
        thd.append(best_thresh)
    print('LFWACC={:.4f} std={:.4f} thd={:.4f}'.format(np.mean(accuracy), np.std(accuracy), np.mean(thd)))

    return np.mean(accuracy), predicts


if __name__ == '__main__':
    _, result = eval_mask(sphere_net().to('cuda'))
    np.savetxt("result.txt", result, '%s')

RuntimeError: ignored

LFW mask evaluation

In [ ]:
from PIL import Image
import numpy as np

from torchvision.transforms import functional as F
import torchvision.transforms as transforms
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

cudnn.benchmark = True


def extractDeepFeature_mask(img, model, is_gray, mask_extract):
    if is_gray:
        transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5,), std=(0.5,))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])
    else:
        transform = transforms.Compose([
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])

    #img, img_ = transform(img), transform(F.hflip(img))
    #img, img_ = img.unsqueeze(0).to('cuda'), img_.unsqueeze(0).to('cuda')
    #ft = torch.cat((model(img), model(img_)), 1)[0].to('cpu')

    img = transform(img)
    img = img.unsqueeze(0).to('cpu')
    mask_extract = mask_extract.unsqueeze(0).to('cpu')
    ft = model(img, mask)[0].to('cpu')
    # ft = torch.cat((model(img, mask), model(img_, mask)), 1)[0].to('cpu')
    return ft


def KFold_mask(n=6000, n_folds=10):
    folds = []
    base = list(range(n))
    for i in range(n_folds):
        test = base[int(i * n / n_folds):int((i + 1) * n / n_folds)]
        train = list(set(base) - set(test))
        folds.append([train, test])
    return folds


def eval_acc_mask(threshold, diff):
    y_true = []
    y_predict = []
    for d in diff:
        same = 1 if float(d[0]) > threshold else 0
        y_predict.append(same)
        y_true.append(int(d[1]))
    y_true = np.array(y_true)
    y_predict = np.array(y_predict)
    accuracy = 1.0 * np.count_nonzero(y_true == y_predict) / len(y_true)
    return accuracy


def find_best_threshold_mask(thresholds, predicts):
    best_threshold = best_acc = 0
    for threshold in thresholds:
        accuracy = eval_acc(threshold, predicts)
        if accuracy >= best_acc:
            best_acc = accuracy
            best_threshold = threshold
    return best_threshold


def eval_mask(model, model_path=None, is_gray=False):
    predicts = []
    # checkpoint = torch.load(model_path)
    # model.load_state_dict(checkpoint['model_state_dict'])
    # model.eval_mask()
    root = 'C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled/'
    with open('C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairs.txt') as f:
        pairs_lines = f.readlines()[1:]

    with torch.no_grad():
        for i in range(6000):
            p = pairs_lines[i].replace('\n', '').split('\t')

            if 3 == len(p):
                sameflag = 1
                name1 = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[1]))
                name2 = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[2]))
            elif 4 == len(p):
                sameflag = 0
                name1 = p[0] + '/' + p[0] + '_' + '{:04}.jpg'.format(int(p[1]))
                name2 = p[2] + '/' + p[2] + '_' + '{:04}.jpg'.format(int(p[3]))
            else:
                raise ValueError("WRONG LINE IN 'pairs.txt! ")

            with open(root + name1, 'rb') as f:
                img1 =  Image.open(f).convert('RGB')
            with open(root + name2, 'rb') as f:
                img2 =  Image.open(f).convert('RGB')
            masks_new = torch.ones(512, 16, 16)
            f1 = extractDeepFeature_mask(img1, model, is_gray, masks_new)
            f2 = extractDeepFeature_mask(img2, model, is_gray, masks_new)

            distance = f1.dot(f2) / (f1.norm() * f2.norm() + 1e-5)
            #predicts.append('{}\t{}\t{}\t{}\n'.format(name1, name2, distance, sameflag))
            predicts.append([distance, sameflag])

    accuracy = []
    thd = []
    folds = KFold(n=6000, n_folds=10)
    thresholds = np.arange(-1.0, 1.0, 0.005)
    #predicts = np.array(map(lambda line: line.strip('\n').split(), predicts))
    predicts = np.array(predicts)
    #print(type(predicts),predicts.shape,predicts[0])

    for idx, (train, test) in enumerate(folds):
        best_thresh = find_best_threshold(thresholds, predicts[train])
        accuracy.append(eval_acc(best_thresh, predicts[test]))
        thd.append(best_thresh)
    print('LFWACC={:.4f} std={:.4f} thd={:.4f}'.format(np.mean(accuracy), np.std(accuracy), np.mean(thd)))

    return np.mean(accuracy), predicts


if __name__ == '__main__':
    _, result = eval_mask(sphere_net().to('cpu'))
    np.savetxt("result.txt", result, '%s')
    print('Complete evaluating')

TypeError: ignored

# Utils

In [ ]:
from __future__ import print_function
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torchvision
import socket
from datetime import datetime

def get_id_label_map(meta_file):
    keys = []
    values = []
    with open(meta_file, 'r') as f:
        for i, class_name in enumerate(f):
            class_name = class_name.strip()
            keys.append(class_name)
            values.append(i)

    id_label_dict = dict(zip(keys, values))
    return id_label_dict

def get_id_labels_map(meta_file_ori, meta_file_new):
    id_label_dict_ori = get_id_label_map(meta_file_ori)
    keys = []
    label_maps = []
    with open(meta_file_new, 'r') as f:
        for i, class_name in enumerate(f):
            class_name = class_name.strip()
            keys.append(class_name)
            # name new_label old_label
            label_maps.append([i, id_label_dict_ori[class_name]])
    id_label_dict_new = dict(zip(keys, label_maps))

    return id_label_dict_new


def generate_img_file_list(img_folder, list_save_path, dsource):
    f = open(list_save_path, 'a+')
    for root, dirs, files in os.walk(img_folder):
        class_name = root.split("/")[-1]
        class_name = class_name.split("\\")[-1]
        for k, file_name in enumerate(files):
            img_file_path = dsource + '/' + class_name + '/' + file_name
            f.write(img_file_path)
            f.write("\n")

    f.close()


def Untransform_VGG(img, mean_bgr):
    img = img.numpy()
    img = img.transpose(1, 2, 0)
    img += mean_bgr
    img = img.astype(np.uint8)
    img = img[:, :, ::-1]
    return img
def Untransform(img, meann, stdd):
    img.to(torch.float32)
    img = img.mul_(stdd[:,None,None]).add_(meann[:,None,None])
    img = img.numpy()
    img = img.transpose(1, 2, 0)
    return img


def imshow(inp, meann, stdd, title=None):
    """Imshow for Tensor."""
    inp = Untransform(inp, meann, stdd)

    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    # plt.show()


def load_state_dict(model, fname):
    pretrained_weight = torch.load(fname)
    own_state = model.state_dict()
    pre_para_list = []
    own_para_list = []
    for p_id, p_name in enumerate(pretrained_weight):
        pre_para_list.append(p_name)

    for p_id, p_name in enumerate(own_state):
        if p_id != 26 and p_id != 27:
            own_para_list.append(p_name)

    param_map = dict(zip(pre_para_list, own_para_list))
    # write into file to check
    f = open('param_map2.txt', 'w')
    f.write('Pretrained \t:\tSiamese \n')
    for key in param_map:
        key_value = key + '\t:\t' + param_map[key]
        f.write(key_value)
        f.write("\n")
    f.close()
    for pre_name in pretrained_weight:
        own_name = param_map[pre_name]
        if own_name in own_state:
            try:
                own_state[own_name].copy_(pretrained_weight[pre_name])
            except Exception:
                raise RuntimeError(
                    'While copying the parameter named {}, whose dimensions in the model are {} and whose ' \
                    'dimensions in the checkpoint are {}.'.format(own_name, own_state[own_name].size(), \
                                                                  pretrained_weight[pre_name].size()))
        else:
            raise KeyError('unexpected key "{}" in state_dict'.format(own_name))

def load_state_dict_bn(model, fname):
    pretrained_weight = torch.load(fname)
    own_state = model.state_dict()

    pre_para_list = []
    own_para_list = []
    for p_id, p_name in enumerate(pretrained_weight):
        pre_para_list.append(p_name)

    for p_id, p_name in enumerate(own_state):
        # conv_s w conv_s b bn_w bn_b bn_runningmean bn_running_var bn_nums_batch
        if p_id != 26 and p_id != 27 and p_id != 28 and p_id != 29 and p_id != 30 and p_id != 31 and p_id != 32:
            own_para_list.append(p_name)

    param_map = dict(zip(pre_para_list, own_para_list))
    # write into file to check
    f = open('param_map2.txt', 'w')
    f.write('Pretrained \t:\tSiamese \n')
    for key in param_map:
        key_value = key + '\t:\t' + param_map[key]
        f.write(key_value)
        f.write("\n")
    f.close()
    for pre_name in pretrained_weight:
        own_name = param_map[pre_name]
        if own_name in own_state:
            try:
                own_state[own_name].copy_(pretrained_weight[pre_name])
            except Exception:
                raise RuntimeError(
                    'While copying the parameter named {}, whose dimensions in the model are {} and whose ' \
                    'dimensions in the checkpoint are {}.'.format(own_name, own_state[own_name].size(), \
                                                                  pretrained_weight[pre_name].size()))
        else:
            raise KeyError('unexpected key "{}" in state_dict'.format(own_name))


def init_vgg_en(model, weight_file, id_label_dict_new):
    pretrained_weight = torch.load(weight_file)
    own_state = model.state_dict()

    pre_para_list = []
    own_para_list = []
    for p_id, p_name in enumerate(pretrained_weight):
        pre_para_list.append(p_name)

    for p_id, p_name in enumerate(own_state):
        own_para_list.append(p_name)

    param_map = dict(zip(own_para_list, pre_para_list))
    ori_id = []
    new_id = []
    for keys, values in id_label_dict_new.items():
        new_id.append(int(values[0]))
        ori_id.append(int(values[1]))

    for own_name in own_state:
        pre_name = param_map[own_name]
        if (pre_name in pretrained_weight) and ('fc8' not in pre_name):
            try:
                own_state[own_name].copy_(pretrained_weight[pre_name])
            except Exception:
                raise RuntimeError(
                    'While copying the parameter named {}, whose dimensions in the model are {} and whose ' \
                    'dimensions in the checkpoint are {}.'.format(own_name, own_state[own_name].size(), \
                                                                  pretrained_weight[pre_name].size()))
        elif (pre_name in pretrained_weight) and ('fc8.weight' in pre_name):
            pre_weight = pretrained_weight[pre_name]  # torch.Size([2622, 4096])
            own_weight = own_state[own_name]  # torch.Size([2000, 4096])
            own_weight[new_id, :] = pre_weight[ori_id, :]

        elif (pre_name in pretrained_weight) and ('fc8.bias' in pre_name):
            pre_bias = pretrained_weight[pre_name]
            own_bias = own_state[own_name]
            own_bias[new_id] = pre_bias[ori_id]

        else:
            raise KeyError('unexpected key "{}" in state_dict'.format(own_name))


def init_sia_mask(model, weight_file, id_label_dict_new):
    pretrained_weight = torch.load(weight_file)
    own_state = model.state_dict()

    pre_para_list = []
    own_para_list = []
    for p_id, p_name in enumerate(pretrained_weight):
        pre_para_list.append(p_name)

    for p_id, p_name in enumerate(own_state):
        own_para_list.append(p_name)

    param_map = dict(zip(own_para_list, pre_para_list))
    ori_id = []
    new_id = []
    for keys, values in id_label_dict_new.items():
        new_id.append(int(values[0]))
        ori_id.append(int(values[1]))

    for own_name in own_state:
        pre_name = param_map[own_name]
        print(own_name, pre_name)
        if (pre_name in pretrained_weight) and ('fc8' not in pre_name):
            try:
                own_state[own_name].copy_(pretrained_weight[pre_name])
            except Exception:
                raise RuntimeError(
                    'While copying the parameter named {}, whose dimensions in the model are {} and whose ' \
                    'dimensions in the checkpoint are {}.'.format(own_name, own_state[own_name].size(), \
                                                                  pretrained_weight[pre_name].size()))
        elif (pre_name in pretrained_weight) and ('fc8.weight' in pre_name):
            pre_weight = pretrained_weight[pre_name]  # torch.Size([2622, 4096])
            own_weight = own_state[own_name]  # torch.Size([2000, 4096])
            own_weight[new_id, :] = pre_weight[ori_id, :]

        elif (pre_name in pretrained_weight) and ('fc8.bias' in pre_name):
            pre_bias = pretrained_weight[pre_name]
            own_bias = own_state[own_name]
            own_bias[new_id] = pre_bias[ori_id]

        else:
            raise KeyError('unexpected key "{}" in state_dict'.format(own_name))

def init_sia_mask_from_vggen(model, weight_file):
    checkpoint = torch.load(weight_file)
    pretrained_weight = checkpoint['model_state_dict']
    own_state = model.state_dict()

    pre_para_list = []
    own_para_list = []
    for p_id, p_name in enumerate(pretrained_weight):
        pre_para_list.append(p_name)

    #print(pre_para_list)

    for p_id, p_name in enumerate(own_state):
        own_para_list.append(p_name)

    param_map = dict(zip(own_para_list, pre_para_list))
    print('Check the correspondency:sia vgg_en\n')
    for own_name in own_state:
        pre_name = param_map[own_name]
        print(own_name, pre_name)
        if pre_name in pretrained_weight:
            try:
                own_state[own_name].copy_(pretrained_weight[pre_name])
            except Exception:
                raise RuntimeError(
                    'While copying the parameter named {}, whose dimensions in the model are {} and whose ' \
                    'dimensions in the checkpoint are {}.'.format(own_name, own_state[own_name].size(), \
                                                                  pretrained_weight[pre_name].size()))
        else:
            raise KeyError('unexpected key "{}" in state_dict'.format(own_name))


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    output_sorted, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res

# def accuracy(output, target, topk=(1,)):
#     with torch.no_grad():
#         maxk = max(topk)
#         batch_size = target.size(0)
#         _, pred = output.topk(maxk, 1, True, True)
#         pred = pred.t()
#         correct = pred.eq(target.view(1, -1).expand_as(pred))
#         res = []
#         for k in topk:
#             correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
#             res.append(correct_k.mul_(100.0 / batch_size))
#         return res


def create_dir(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)


def get_run_name():
    """ A unique name for each run """
    return datetime.now().strftime(
        '%b%d-%H-%M-%S') + '_' + socket.gethostname()

In [ ]:
pip install datetime

Note: you may need to restart the kernel to use updated packages.


# Extract mask dictionary

In [ ]:
from __future__ import print_function
from __future__ import division
import argparse
import os
import time
import gc


import torch
import torchvision
import torch.utils.data
import torch.optim
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import numpy as np
import sklearn.preprocessing
import matplotlib.pyplot as plt
cudnn.benchmark = True

# import _init_paths
# import models.net_sia as net_sia
# import datasets.dataset_pair as dset
# import lfw_eval
# import layer
# import utils


plt.switch_backend('agg')
parser = argparse.ArgumentParser(description='PyTorch CosFace')
# DATA
parser.add_argument('--root_path', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled',
                    help='path to root path of images')
parser.add_argument('--train_list', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain.txt', help='path to training list')
parser.add_argument('--batch_size', type=int, default=6,
                    help='input batch size for training (default: 512)')
parser.add_argument('--is_gray', type=bool, default=False,
                    help='Transform input image to gray or not  (default: False)')
# Network
parser.add_argument('--weight_model', type=str, default='')
parser.add_argument('--save_path', type=str, default='masks/mean/d12/',
                    help='path to save checkpoint')
parser.add_argument('--no_cuda', type=bool, default=False,
                    help='disables CUDA training')
parser.add_argument('--workers', type=int, default=0,
                    help='how many workers to load data')
parser.add_argument('--gpus', type=str, default='4,5,6,7')
parser.add_argument('--ngpus', type=int, default=4)
parser.add_argument('--network', type=str, default='')
parser.add_argument('-f', required=False)

args = parser.parse_args()
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpus
args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if args.cuda else "cpu")


def main():
    # --------------------------------------model----------------------------------------
    model = LResNet50E_IR_Sia(is_gray=args.is_gray)
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpus

    model = model.to(device)

    args.run_name = get_run_name()  # Dec25-14-53-43_lingxuesong-PC0
    output_dir = os.path.join(args.save_path, args.network)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # ------------------------------------load image---------------------------------------
    if args.is_gray:
        train_transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5,), std=(0.5,))
        ])  # gray
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])
    train_loader = torch.utils.data.DataLoader(
        ImageList_pair(root=args.root_path, fileList=args.train_list,
                  transform=train_transform),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True, drop_last=True)

    print('length of train Database: ' + str(len(train_loader.dataset)) + ' Batches: ' + str(len(train_loader)))
    # ----------------------------------------train----------------------------------------
    extract(train_loader, model, args.weight_model, output_dir)
    print('Finished Extracting')


def extract(val_loader, model, model_path, output_dir):
    # ckpt = torch.load(model_path)
    # model.load_state_dict(ckpt['model_state_dict'])

    model.eval()
    OUT_SUM = np.zeros([512*256])
    count_p = 0
    with torch.no_grad():
        for batch_idx, (data, data_occ, pair_files, _) in enumerate(val_loader, 1):


            data, data_occ = data.to(device), data_occ.to(device)
            # compute output
            _, _, _, _, _, out = model(data, data_occ)

            print('Extracting for No.:{} batch...shape is:{}'.format(batch_idx + 1, out.shape))

            assert out.shape[0] == len(pair_files)

            for i, pair_file in enumerate(pair_files):
                out_cpu = out[i].to('cpu')
                out_cpu = out_cpu.view(-1,1)
                min_max_scaler = sklearn.preprocessing.MinMaxScaler()
                out_cpu = min_max_scaler.fit_transform(out_cpu)
                OUT_SUM = OUT_SUM + out_cpu.flatten()
                count_p = count_p + 1


    OUT_MEAN = OUT_SUM / count_p

    feature_file = os.path.join(output_dir, 'mean_mask.txt')
    np.savetxt(feature_file, OUT_MEAN)



if __name__ == '__main__':
    print(args)
    main()

Namespace(root_path='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled', train_list='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain.txt', batch_size=6, is_gray=False, weight_model='', save_path='masks/mean/d12/', no_cuda=False, workers=0, gpus='4,5,6,7', ngpus=4, network='', f='C:\\Users\\Admin\\AppData\\Roaming\\jupyter\\runtime\\kernel-7e835303-42a1-4944-9881-91eb40ded7a1.json', cuda=False)
length of train Database: 100 Batches: 16
Extracting for No.:2 batch...shape is:torch.Size([6, 512, 16, 16])
Extracting for No.:3 batch...shape is:torch.Size([6, 512, 16, 16])
Extracting for No.:4 batch...shape is:torch.Size([6, 512, 16, 16])
Extracting for No.:5 batch...shape is:torch.Size([6, 512, 16, 16])
Extracting for No.:6 batch...shape is:torch.Size([6, 512, 16, 16])
Extracting for No.:7 batch...shape is:torch.Size([6, 512, 16, 16])
Extracting for No.:8 batch...shape is:torch.Size([6, 512, 16, 16])
Extracting for No.:9

In [ ]:
# Import label encoder
from sklearn import preprocessing
import tensorflow as tf
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
labelList = []
label_unique = []
file = open('C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain.txt', 'r')
for line in file.readlines():
  list_new = line.strip().split('\t')
  imgPath, imgOccPath, label_new = list_new[1], list_new[2], list_new[0]
  label_new = label_new[:-14]
  labelList.append(label_new)

# print(labelList)
label_2 = label_encoder.fit_transform(labelList)
label_3 = label_2
print(label_3)
label_2 = torch.Tensor(label_2)
label_2 = label_2.type(torch.int64)
label_2 = torch.unsqueeze(label_2, 0)[0]
label_2 = torch.unique(label_2)

print(label_2)
# print(label_unique)

[ 44   3   3   4   5   5   5   5   5   5   5   5   5   5   5   5   6   6
   7   8   8   9   9  11  13  13  14  15  15  16  16  16  16  16  16  16
  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  17
  17  19  19  19  19  20  20  20  20  20  20  20  20  20  20  20  20  20
  21  21  22  22  22  23  23  23  23  24  25  25  81 107  36  60  47  69
 114 120 141  51 104 167 179  28  73 140 171 177  35  80 106 143 184 199
  50  54  62  66  75  77 154 191  88 105 139 185   1  32  42  65 133 146
 190 198  57  68  91  97 102 122 123 174 176 196 117 126 128 148 178  83
  85  89 147 162 183 186 187 197   2  56  92 135 152 158 163 192  43  76
  87  93 136 144 145 155 168 194   0  30  38  94 115 138 149 157 159 161
 173  18  45  82 108 109 131 132 142 153 156 160 165  12  31  48  52  86
  90 124 125 195  59  78  79  84 101 112 121 182  33  34  41  46  58  72
  95  96 103 113 116 119 127 150 151 169 181 189 193  10  49  53  64  74
  98 110 111 129 130 134 137 166 172  29  67  99 11

# Main dictionary

In [ ]:
from __future__ import print_function
from __future__ import division
import argparse
import os
import time
import gc


import torch
import torchvision
import torch.utils.data
import torch.optim
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import torch.backends.cudnn as cudnn
import numpy as np
import matplotlib.pyplot as plt
cudnn.benchmark = True

# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# import _init_paths
# import models.net_sia as net_sia
# import datasets.dataset_pair as dset
# import layer
# import utils

plt.switch_backend('agg')
configurations = {
    1: dict(
        lr=1.0e-1,
        step_size=[7500, 15000, 22500],  # "lr_policy: step" [15, 30, 45, 50]
        epochs=30,
    ),
    2: dict(
        lr=1.0e-1,
        step_size=[10000, 17500, 22500],  # [20, 35, 45, 50] MultiStepLR
        epochs=30,
    )
}


# Training settings
parser = argparse.ArgumentParser(description='PyTorch CosFace')

# DATA
parser.add_argument('--root_path', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled',
                    help='path to root path of images')
parser.add_argument('--train_list', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain_20.txt',help='path to training pair list')
parser.add_argument('--valid_list', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevVal_10.txt',help='path to validating pair list')
parser.add_argument('--batch_size', type=int, default=1,
                    help='input batch size for training (default: 512)')
parser.add_argument('--is_gray', type=bool, default=False,
                    help='Transform input image to gray or not  (default: False)')
# Network
parser.add_argument('--weight_model', type=str, default='C:/Jun21-22-24-34/CosFace_10_checkpoint.pth')
parser.add_argument('--weight_fc', type=str, default='C:/Jun21-22-24-34/CosFace_10_checkpoint_classifier.pth')
parser.add_argument('--resume', type=str, default='')
parser.add_argument('--resume_fc', type=str, default='')
parser.add_argument('--s_weight', type=float, default=10.0)
# Classifier
parser.add_argument('--num_class', type=int, default=200,
                    help='number of people(class)')
parser.add_argument('--classifier_type', type=str, default='MCP',
                    help='Which classifier for train. (MCP, AL, L)')
# LR policy
parser.add_argument('--epochs', type=int, default=30,
                    help='number of epochs to train (default: 30)')
parser.add_argument('--lr', type=float, default=0.1,
                    help='learning rate (default: 0.1)')
parser.add_argument('--step_size', type=list, default=None,
                    help='lr decay step')
parser.add_argument('--momentum', type=float, default=0.9,
                    help='SGD momentum (default: 0.9)')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    metavar='W', help='weight decay (default: 0.0005)')
parser.add_argument('-c', '--config', type=int, default=1, choices=configurations.keys(),
                    help='the number of settings and hyperparameters used in training')
# Common settings
parser.add_argument('--log_interval', type=int, default=100,
                    help='how many batches to wait before logging training status')
parser.add_argument('--save_path', type=str, default='',
                    help='path to save checkpoint')
parser.add_argument('--no_cuda', type=bool, default=False,
                    help='disables CUDA training')
parser.add_argument('--workers', type=int, default=0,
                    help='how many workers to load data')
parser.add_argument('--gpus', type=str, default='4,5,6,7')
parser.add_argument('--ngpus', type=int, default=4)
parser.add_argument('--d_name', type=str, default='')
parser.add_argument('-f', required=False)

args = parser.parse_args()
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpus
args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if args.cuda else "cpu")


def main():
    # --------------------------------------model----------------------------------------
    model = LResNet50E_IR_Sia(is_gray=args.is_gray)
    model_eval = LResNet50E_IR_Sia(is_gray=args.is_gray)
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpus
    # 512 is dimension of feature
    classifier = {
        'MCP': MarginCosineProduct(512, args.num_class),
        'AL' : AngleLinear(512, args.num_class),
        'L'  : torch.nn.Linear(512, args.num_class, bias=False)
    }[args.classifier_type]

    # classifier.load_state_dict(torch.load(args.weight_fc))

    print(os.environ['CUDA_VISIBLE_DEVICES'], args.cuda)



    # pretrained = torch.load(args.weight_model)
    # print(pretrained)
    # pretrained_dict = pretrained['model_state_dict']
    model_dict = model.state_dict()
    model_eval_dict = model_eval.state_dict()
    # for k, v in pretrained_dict.items():
    #     if k in model_dict:
    #         model_dict[k].copy_(v)

    # del pretrained
    # del pretrained_dict
    if args.resume:
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state_dict'])
        # classifier ckpt only save model info
        classifier.load_state_dict(torch.load(args.resume_fc))
    print(model)
    model = torch.nn.DataParallel(model).to(device)
    model_eval = model_eval.to(device)
    classifier = classifier.to(device)


    args.run_name = get_run_name()
    output_dir = os.path.join(args.save_path, args.run_name.split("_")[0])
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # ------------------------------------load image---------------------------------------
    if args.is_gray:
        train_transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5,), std=(0.5,))
        ])  # gray
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])
        valid_transform = transforms.Compose([
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])
    train_loader = torch.utils.data.DataLoader(
        ImageList_pair(root=args.root_path, fileList=args.train_list,
                  transform=train_transform),
        batch_size=args.batch_size, shuffle=True,
        num_workers=args.workers, pin_memory=False, drop_last=True)
    val_loader = torch.utils.data.DataLoader(
        ImageList_pair(root=args.root_path, fileList=args.valid_list,
                  transform=valid_transform),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=False, drop_last=False)

    print('length of train Database: ' + str(len(train_loader.dataset)) + ' Batches: ' + str(len(train_loader)))
    print('length of valid Database: ' + str(len(val_loader.dataset)) + ' Batches: ' + str(len(val_loader)))
    print('Number of Identities: ' + str(args.num_class))
    # Get a batch of training data, (img, img_occ, label)
    '''
    inputs, inputs_occ, imgPair, targets = next(iter(train_loader))
    out = torchvision.utils.make_grid(inputs)
    out_occ = torchvision.utils.make_grid(inputs_occ)

    mean = torch.tensor((0.5,0.5,0.5), dtype=torch.float32)
    std = torch.tensor((0.5,0.5,0.5), dtype=torch.float32)
    utils.imshow(out, mean, std, title=str(targets))
    plt.savefig(output_dir + '/train.png')
    utils.imshow(out_occ, mean, std, title=str(targets))
    plt.savefig(output_dir + '/train_occ.png')
    '''
    #---------------------------------------params setting-----------------------------------
    for name, param in model.named_parameters():
        if 'layer' in name or 'conv1' in name or 'bn1' in name or 'prelu1' in name:
            param.requires_grad = False
        else:
            param.requires_grad = True

    print("Params to learn:")
    params_to_update = []
    params_to_stay = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            if 'sia' in name:
                params_to_update.append(param)
                print("Update \t", name)
            else:
                params_to_stay.append(param)
                print("Stay \t", name)

    for name, param in classifier.named_parameters():
        param.requires_grad = True
        params_to_stay.append(param)
        print("Stay \t", name)
    #--------------------------------loss function and optimizer-----------------------------
    cfg = configurations[args.config]
    criterion = torch.nn.CrossEntropyLoss().to(device)
    criterion2 = torch.nn.L1Loss(reduction='mean').to(device)
    optimizer = torch.optim.SGD([
                                    {'params': params_to_stay, 'lr': 0, 'weight_decay': 0, 'momentum': 0},
                                    {'params': params_to_update}
                                ],
                                lr=cfg['lr'],
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)
    start_epoch = 1
    if args.resume:
        optimizer.load_state_dict(checkpoint['optim_state_dict'])
        start_epoch = checkpoint['epoch']
        del checkpoint
    # ----------------------------------------train----------------------------------------
    save_ckpt(model, 0, optimizer, output_dir + '/CosFace_0_checkpoint.pth') # Not resumed, pretrained~
    for epoch in range(start_epoch, cfg['epochs'] + 1):
        train(train_loader, model, classifier, criterion, criterion2, optimizer, epoch, cfg['step_size'], cfg['lr'])
        save_ckpt(model, epoch, optimizer, output_dir + '/CosFace_' + str(epoch) + '_checkpoint.pth')
        torch.save(classifier.state_dict(), output_dir + '/CosFace_' + str(epoch) + '_checkpoint_classifier.pth')
        print('Validating on valid set...')
        valid(val_loader, model_eval, output_dir + '/CosFace_' + str(epoch) + '_checkpoint.pth', classifier, criterion, criterion2)
    print('Finished Training')

label_2 = label_2.to(device)

def valid(val_loader, model, model_path, classifier, criterion, criterion2):
    ckpt = torch.load(model_path)
    model.load_state_dict(ckpt['model_state_dict'])
    model.eval()

    top1_clean = AverageMeter()
    top1_occ = AverageMeter()
    losses_clean = AverageMeter()
    losses_occ = AverageMeter()
    sia_losses = AverageMeter()

    time_curr = time.time()
    time_curr0 = time.time()


    with torch.no_grad():
        for batch_idx, (data, data_occ, _, target) in enumerate(val_loader, 1):

            data, data_occ = data.to(device), data_occ.to(device)
            f_masked, focc_masked, output, output_occ, _, _ = model(data, data_occ)
            # f_masked_diff = torch.abs(f_masked - focc_masked)
            # temp = torch.sum(f_masked_diff) / (f_masked.size(0) * 512 * 7 * 6)

            sia_loss = criterion2(focc_masked, f_masked) # / (f_masked.size(0) * 512 * 7 * 6)
            sia_losses.update(sia_loss.item(),data.size(0))

            score = torch.mm(output, classifier.weight.t())
            prec1 = accuracy(score.data, target.data)
            top1_clean.update(prec1[0].item(),data.size(0))

            score_occ = torch.mm(output_occ, classifier.weight.t())
            prec1_occ = accuracy(score_occ.data, target.data)
            top1_occ.update(prec1_occ[0].item(),data.size(0))

            target_idx = []
            label_2_np = label_2.numpy()
            # label_unique_np = label_unique.numpy()
            for k in target:
              target_idx.append(np.where(label_2_np == k[0].item())[0][0])

            target_idx = torch.from_numpy(np.array(target_idx))
            output = classifier(output, label_2)
            loss_clean = criterion(output, target_idx)
            output_occ = classifier(output_occ, label_2)
            loss_occ = criterion(output_occ, target_idx)

            loss = 0.5 * loss_clean + 0.5 * loss_occ + 10 * sia_loss
            #print(loss_clean.item(), loss_occ.item(), sia_loss.item())
            if np.isnan(float(loss.item())):
                raise ValueError('loss is nan while validating')
            losses_clean.update(loss_clean.item(),data.size(0))
            losses_occ.update(loss_occ.item(),data.size(0))


            if batch_idx % 100 == 0:
                time_used = time.time() - time_curr
                print_with_time(
                        'Valid : [{}/{} ({:.0f}%)], SiaLoss: {:.6f}, CleanLoss: {:.6f}, OccLoss: {:.6f}, Prec: {:.4f}, Prec_occ: {:.4f}, Elapsed time: {:.4f}s({} iters)'.format(
                        batch_idx * len(data), len(val_loader.dataset), 100. * batch_idx / len(val_loader),
                        sia_losses.avg, losses_clean.avg, losses_occ.avg, top1_clean.avg, top1_occ.avg, time_used, 100)
                )
                time_curr = time.time()

        # end of validation print
        time_used = time.time() - time_curr0
        print_with_time(
                'Valid Summary : SiaLoss: {:.6f}, CleanLoss: {:.6f}, OccLoss: {:.6f}, Prec: {:.4f}, Prec_occ: {:.4f}, Elapsed time: {:.4f}s'.format(
                    sia_losses.avg, losses_clean.avg, losses_occ.avg, top1_clean.avg, top1_occ.avg, time_used)
        )


def train(train_loader, model, classifier, criterion, criterion2, optimizer, epoch, step_size, base_lr):

    for name, module in model.module.named_children():
        if name in ['sia']:
            module.train(True)
        else:
            module.train(False)
    print_with_time('Epoch {} start training'.format(epoch))
    time_curr = time.time()
    time_curr0 = time.time()
    loss_display = 0.0

    top1_clean = AverageMeter()
    top1_occ = AverageMeter()
    losses_clean = AverageMeter()
    losses_occ = AverageMeter()
    sia_losses = AverageMeter()

    label_encoder = preprocessing.LabelEncoder()
    labelList = []
    file = open('C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain.txt', 'r')
    for line in file.readlines():
      list_new = line.strip().split('\t')
      imgPath, imgOccPath, label_new = list_new[1], list_new[2], list_new[0]
      label_new = label_new[:-14]
      labelList.append(label_new)



    for batch_idx, (data, data_occ, _, target) in enumerate(train_loader, 1):

        iteration = (epoch - 1) * len(train_loader) + batch_idx
        adjust_learning_rate(optimizer, iteration, step_size, base_lr)
        data, data_occ, target = data.to(device), data_occ.to(device), target.to(device)
        data, data_occ = data.to(device), data_occ.to(device)
        f_masked, focc_masked, output, output_occ, f_diff, out = model(data, data_occ)

        #----------------------------------------Calculate statistics--------------------------------------#
        sia_loss = criterion2(focc_masked, f_masked) #/ (f_masked.size(0) * 512 * 7 * 6)
        sia_losses.update(sia_loss.item(),data.size(0))
        #f_masked_diff = torch.abs(f_masked - focc_masked)
        #temp = torch.sum(f_masked_diff) / (f_masked.size(0) * 512 * 7 * 6)
        #print(temp.item(),sia_loss.item(), f_masked.shape)

        score = torch.mm(output, classifier.weight.t())
        prec1 = accuracy(score.data, target.data)
        top1_clean.update(prec1[0].item(),data.size(0))

        score_occ = torch.mm(output_occ, classifier.weight.t())
        prec1_occ = accuracy(score_occ.data, target.data)
        top1_occ.update(prec1_occ[0].item(),data.size(0))

        output = classifier(output, label_2)
        print(output.argmax())
        target_idx = []
        label_2_np = label_2.numpy()

        for k in target:
          target_idx.append(np.where(label_2_np == k[0].item())[0][0])

        target_idx = torch.from_numpy(np.array(target_idx))
        loss_clean = criterion(output, target_idx)
        losses_clean.update(loss_clean.item(),data.size(0))

        output_occ = classifier(output_occ, label_2)
        print(output_occ.argmax())
        loss_occ = criterion(output_occ, target_idx)
        losses_occ.update(loss_occ.item(),data.size(0))
        #---------------------------------------------Backward----------------------------------------------#
        loss = 0.5 * loss_clean + 0.5 * loss_occ + 10 * sia_loss
        print('Target: ', target, 'Target idx: ', target_idx, 'Loss clean: ', loss_clean.item(), 'Loss occ: ', loss_occ.item(), 'Sia loss ', sia_loss.item())
        if np.isnan(float(loss.item())):
            raise ValueError('loss is nan while validating')
        loss_display += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % 1600 == 0:
            print(out[0,0,:,:])
        if batch_idx % 100 == 0:
            time_used = time.time() - time_curr
            loss_display /= 100

            print_with_time(
                    'Train Epoch: {} [{}/{} ({:.0f}%)]{}, SiaLoss: {:.6f}, ClsLoss: [{:.6f}/{:.6f}/{:.6f}], Prec: {:.4f}, Prec_occ: {:.4f}, Elapsed time: {:.4f}s, LR: {:.5f}/{:.5f}({} iters) INFO: [{}/{}/{}]'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                    iteration, sia_losses.avg, losses_clean.avg, losses_occ.avg, loss_display, top1_clean.avg, top1_occ.avg,
                    time_used, optimizer.param_groups[0]['lr'],optimizer.param_groups[1]['lr'],100, 'net2', args.s_weight,args.d_name)
            )
            time_curr = time.time()
            loss_display = 0.0
    time_used = time.time() - time_curr0
    print_with_time(
            'Train Summary : SiaLoss: {:.6f}, ClsLoss: [{:.6f}/{:.6f}], Prec: {:.4f}, Prec_occ: {:.4f}, Elapsed time: {:.4f}s, INFO: [{}/{}]'.format(
                sia_losses.avg, losses_clean.avg, losses_occ.avg, top1_clean.avg, top1_occ.avg, time_used, 'net2', args.s_weight)
    )


def print_with_time(string):
    print(time.strftime("%Y-%m-%d %H:%M:%S ", time.localtime()) + string)


def adjust_learning_rate(optimizer, iteration, step_size, base_lr):
    """Sets the learning rate to the initial LR decayed by 10 each step size"""
    if iteration in step_size:
        lr = base_lr * (0.1 ** (step_size.index(iteration) + 1))
        print_with_time('Adjust base learning rate to {}'.format(lr))
        optimizer.param_groups[0]['lr'] = 0
        optimizer.param_groups[1]['lr'] = lr
    else:
        pass


def save_ckpt(model, epoch, optimizer, save_name):
    """Save checkpoint"""
    if isinstance(model, torch.nn.DataParallel):
        model = model.module

    torch.save({
        'epoch': epoch,
        'optim_state_dict': optimizer.state_dict(),
        'model_state_dict': model.state_dict()}, save_name)


if __name__ == '__main__':
    print(args)
    main()

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Target:  tensor([[16]], dtype=torch.int32) Target idx:  tensor([16]) Loss clean:  0.0005108005134388804 Loss occ:  0.0001998939987970516 Sia loss  0.11874257773160934
tensor(4)
tensor(4)
Target:  tensor([[4]], dtype=torch.int32) Target idx:  tensor([4]) Loss clean:  0.08003922551870346 Loss occ:  0.05267355963587761 Sia loss  0.10024084150791168
tensor(25)
tensor(25)
Target:  tensor([[25]], dtype=torch.int32) Target idx:  tensor([25]) Loss clean:  0.006283053662627935 Loss occ:  0.02715027891099453 Sia loss  0.1149541586637497
tensor(19)
tensor(19)
Target:  tensor([[19]], dtype=torch.int32) Target idx:  tensor([19]) Loss clean:  0.006114941090345383 Loss occ:  0.00442715547978878 Sia loss  0.11525595188140869
tensor(5)
tensor(5)
Target:  tensor([[5]], dtype=torch.int32) Target idx:  tensor([5]) Loss clean:  0.0005025792634114623 Loss occ:  0.0010966007830575109 Sia loss  0.08875206112861633
tensor(147)
tensor(147)
Target:  tensor

KeyboardInterrupt: ignored

In [ ]:
label_2[0]

tensor([ 28,  17,  83,  60, 139,  66,  94,  78, 148,  27,  73,  93, 161,   7,
          8,  84,  97, 146, 106, 157, 198,  67, 100,  19,  44,  31,  53, 108,
        114, 135,  35,  96, 166, 178,  10,  57, 134, 170, 176,  18,  65,  99,
        137, 183, 199,  34,  38,  46,  50,  59,  62, 151, 190,  75,  98, 133,
        184,   1,  14,  25,  49, 127, 141, 189, 197,  41,  52,  79,  87,  92,
        116, 117, 173, 175, 195, 111, 120, 122, 143, 177,  69,  71,  76, 142,
        160, 182, 185, 186, 196,   2,  40,  80, 129, 149, 155, 162, 191,  26,
         61,  74,  81, 130, 138, 140, 152, 167, 193,   0,  12,  21,  82, 109,
        132, 144, 154, 156, 159, 172,   5,  29,  68, 101, 102, 125, 126, 136,
        150, 153, 158, 164,   4,  13,  32,  36,  72,  77, 118, 119, 194,  43,
         63,  64,  70,  91, 105, 115, 181,  15,  16,  24,  30,  42,  56,  85,
         86,  95, 107, 110, 113, 121, 145, 147, 168, 180, 188, 192,   3,  33,
         37,  48,  58,  88, 103, 104, 123, 124, 128, 131, 165, 1

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
print(input)

tensor([4, 0, 3])
tensor([[ 0.9839,  2.7239,  0.7926,  0.6511,  0.4204],
        [ 3.1953, -1.6550,  0.7018, -0.4427,  0.2859],
        [ 0.3871, -0.5177, -3.0814, -0.6760,  0.4364]], requires_grad=True)


In [ ]:
import torch
import torchvision
import torchvision.transforms as T
from PIL import Image
occ_img_path = 'C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled/Aaron_Eckhart/Aaron_Eckhart_0001.jpg'
occ_img = Image.open(occ_img_path)
occ_img = torchvision.transforms.Resize((128,128))(occ_img)
occ_img.show()

# Main webface mask

In [ ]:
from __future__ import print_function
from __future__ import division
import argparse
import os
import time

import torch
import torch.utils.data
import torch.optim
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

cudnn.benchmark = True


# import _init_paths
# import net_msk
# import datasets.dataset_pair_msk as dset
# import lfw_eval_msk
# import lfw_occ_eval_msk
# import layer
# import utils


# Training settings
parser = argparse.ArgumentParser(description='PyTorch CosFace')

# DATA
parser.add_argument('--root_path', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/MLFW/aligned/',
                    help='path to root path of images')
parser.add_argument('--lfw_path', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled/',
                    help='path to root path of images')
parser.add_argument('--database', type=str, default='WebFace')
parser.add_argument('--train_list', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain.txt',
                    help='path to training list')
parser.add_argument('--batch_size', type=int, default=5,
                    help='input batch size for training (default: 256)')
parser.add_argument('--is_gray', type=bool, default=False,
                    help='Transform input image to gray or not  (default: False)')
# Network
parser.add_argument('--weight_model', type=str, default='')
parser.add_argument('--weight_fc', type=str, default='')
parser.add_argument('--resume', type=str, default='')
parser.add_argument('--resume_fc', type=str, default='')
parser.add_argument('--clean_mask_flag', type=int, default=0)
# Classifier
parser.add_argument('--num_class', type=int, default=None,
                    help='number of people(class)')
parser.add_argument('--classifier_type', type=str, default='MCP')
# LR policy
parser.add_argument('--epochs', type=int, default=10,
                    help='number of epochs to train (default: 30)')
parser.add_argument('--lr', type=float, default=0.001,
                    help='learning rate (default: 0.1)')
parser.add_argument('--lr_freeze', type=float, default=0.1)
parser.add_argument('--step_size', type=list, default=None,
                    help='lr decay step')
parser.add_argument('--momentum', type=float, default=0.9,
                    help='SGD momentum (default: 0.9)')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    metavar='W', help='weight decay (default: 0.0005)')
# Common settings
parser.add_argument('--log_interval', type=int, default=100,
                    help='how many batches to wait before logging training status')
parser.add_argument('--save_path', type=str, default='checkpoint/',
                    help='path to save checkpoint')
parser.add_argument('--no_cuda', type=bool, default=False,
                    help='disables CUDA training')
parser.add_argument('--workers', type=int, default=0,
                    help='how many workers to load data')
parser.add_argument('--gpus', type=str, default='0,1,2,3')
parser.add_argument('-f', required=False)

args = parser.parse_args()
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpus
args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if args.cuda else "cpu")

if args.database is 'WebFace':
    args.num_class = 100
    args.step_size = [50, 100, 150] # 452722  [5 10] 0.001 begin
else:
    raise ValueError("NOT SUPPORT DATABASE! ")


def main():
    # --------------------------------------model----------------------------------------
    model = LResNet50E_IR_netmask(is_gray=args.is_gray)
    model_eval = LResNet50E_IR_netmask(is_gray=args.is_gray)

    args.run_name = get_run_name()
    output_dir = os.path.join(args.save_path, args.run_name.split("_")[0])
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 512 is dimension of feature
    classifier = {
        'MCP': MarginCosineProduct(512, args.num_class),
        'AL' : AngleLinear(512, args.num_class),
        'L'  : torch.nn.Linear(512, args.num_class, bias=False)
    }[args.classifier_type]

    # load pretrained weight
    # pretrained = torch.load(args.weight_model)
    # model.load_state_dict(pretrained['model_state_dict'])
    # model_eval.load_state_dict(pretrained['model_state_dict'])
    # classifier.load_state_dict(torch.load(args.weight_fc))
    # del pretrained
    start_epoch = 1
    if args.resume:
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state_dict'])
        start_epoch = checkpoint['epoch']
        classifier.load_state_dict(torch.load(args.resume_fc))
        print("Resume from epoch: {}".format(start_epoch))

    model = torch.nn.DataParallel(model).to(device)
    model_eval = model_eval.to(device)
    classifier = classifier.to(device)
    print(model)
    model.module.save(output_dir + '/CosFace_0_checkpoint.pth')

    # ------------------------------------load image---------------------------------------
    if args.is_gray:
        train_transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5,), std=(0.5,))
        ])  # gray
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
        ])
    train_loader = torch.utils.data.DataLoader(
        ImageList_pairmask(root=args.root_path, fileList=args.train_list,
                  transform=train_transform),
        batch_size=args.batch_size, shuffle=True,
        num_workers=args.workers, pin_memory=True, drop_last=True)

    print('length of train Database: ' + str(len(train_loader.dataset)) + '  Batches: ' + str(len(train_loader)))
    print('Number of Identities: ' + str(args.num_class))

    # --------------------------------params setting-----------------------------
    print("Params to learn:")
    params_to_update = []
    params_to_freeze = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            if 'fc' in name: # or 'layer4' in name:
                params_to_update.append(param)
                print("Update \t", name)
            else:
                params_to_freeze.append(param)
                print("Freeze \t", name)
    for name, param in classifier.named_parameters():
        param.requires_grad = True
        params_to_update.append(param)
        print("Update \t", name)

    # --------------------------------loss function and optimizer-----------------------------
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD([
                                    {'params': params_to_freeze, 'lr':args.lr * args.lr_freeze},
                                    {'params': params_to_update}
                                ],
                                lr=args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)
    if args.resume:
        optimizer.load_state_dict(checkpoint['optim_state_dict'])
    # ----------------------------------------train----------------------------------------
    save_ckpt(model, 0, optimizer, output_dir + '/CosFace_0_checkpoint.pth')

    # eval_mask(model_eval, output_dir + '/CosFace_0_checkpoint.pth', args.is_gray)
    for epoch in range(start_epoch, args.epochs + 1):
        train(train_loader, model, classifier, criterion, optimizer, epoch)
        save_ckpt(model, epoch, optimizer, output_dir + '/CosFace_' + str(epoch) + '_checkpoint.pth')
        torch.save(classifier.state_dict(), output_dir + '/CosFace_' + str(epoch) + '_checkpoint_classifier.pth')
        # eval_mask(model_eval, output_dir + '/CosFace_' + str(epoch) + '_checkpoint.pth', args.is_gray)
        # eval_mask(model_eval, args.lfw_path, output_dir + '/CosFace_' + str(epoch) + '_checkpoint.pth', args.is_gray)
    print('Finished Training')


def train(train_loader, model, classifier, criterion, optimizer, epoch):
    model.train()
    print_with_time('Epoch {} start training'.format(epoch))
    time_curr = time.time()
    loss_display = 0.0
    losses_clean = AverageMeter()
    losses_occ = AverageMeter()

    # mask_ones = torch.ones(512, 16, 16)
    mask_ones = torch.ones(512, 16, 16)

    for batch_idx, (data, data_occ, _, masks, target) in enumerate(train_loader, 1):
        iteration = (epoch - 1) * len(train_loader) + batch_idx
        adjust_learning_rate(optimizer, iteration, args.step_size)

        mask1 = mask_ones.expand(data.size(0), -1, -1, -1)

        data, data_occ, target = data.to(device), data_occ.to(device), target.to(device)
        masks = masks.to(device)
        # compute output
        if args.clean_mask_flag == 1:
            output = model(data, masks)
        else:
            mask1 = mask1.to(device)
            output = model(data, mask1)
        output = classifier(output, target)
        target = torch.tensor(target)
        target = torch.argmax(target, axis=1)
        loss_clean = criterion(output, target)

        output_occ = model(data_occ, masks)
        output_occ = classifier(output_occ, target)
        loss_occ = criterion(output_occ, target)

        losses_clean.update(loss_clean.item(),data.size(0))
        losses_occ.update(loss_occ.item(),data.size(0))

        loss = 0.5 * loss_clean + 0.5 * loss_occ
        loss_display += loss.item()
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            time_used = time.time() - time_curr
            loss_display /= args.log_interval
            if args.classifier_type is 'MCP':
                INFO = ' Freeze: {:.4f}, C_M_Flag: {} [INFO:WebMsk]'.format(args.lr_freeze, args.clean_mask_flag)
            elif args.classifier_type is 'AL':
                INFO = ' lambda: {:.4f}'.format(classifier.lamb)
            else:
                INFO = ''
            print_with_time(
                    'Train Epoch: {} [{}/{} ({:.0f}%)]{}, Loss: {:.6f}/{:.6f}/{:.6f}, Elapsed time: {:.4f}s, LR: {:.6f}/{:.6f}({} iters)'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                    iteration, losses_clean.avg, losses_occ.avg, loss_display, time_used,
                    optimizer.param_groups[0]['lr'], optimizer.param_groups[1]['lr'],args.log_interval) + INFO
            )
            time_curr = time.time()
            loss_display = 0.0
        torch.cuda.empty_cache()


def print_with_time(string):
    print(time.strftime("%Y-%m-%d %H:%M:%S ", time.localtime()) + string)


def adjust_learning_rate(optimizer, iteration, step_size):
    """Sets the learning rate to the initial LR decayed by 10 each step size"""
    if iteration in step_size:
        lr = args.lr * (0.1 ** (step_size.index(iteration) + 1))
        print_with_time('Adjust learning rate to {}'.format(lr))
        optimizer.param_groups[0]['lr'] = args.lr_freeze * lr
        optimizer.param_groups[1]['lr'] = lr
    else:
        pass

def save_ckpt(model, epoch, optimizer, save_name):
    """Save checkpoint"""
    if isinstance(model, torch.nn.DataParallel):
        model = model.module

    torch.save({
        'epoch': epoch,
        # 'arch': self.model.__class__.__name__,
        'optim_state_dict': optimizer.state_dict(),
        'model_state_dict': model.state_dict()}, save_name)

if __name__ == '__main__':
    print(args)
    main()

Namespace(batch_size=5, classifier_type='MCP', clean_mask_flag=0, cuda=False, database='WebFace', epochs=10, f='C:\\Users\\Admin\\AppData\\Roaming\\jupyter\\runtime\\kernel-bcd6ce70-f26c-4b15-ad01-460ec03bdd7c.json', gpus='0,1,2,3', is_gray=False, lfw_path='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled/', log_interval=100, lr=0.001, lr_freeze=0.1, momentum=0.9, no_cuda=False, num_class=100, resume='', resume_fc='', root_path='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/MLFW/aligned/', save_path='checkpoint/', step_size=[50, 100, 150], train_list='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain.txt', weight_decay=0.0005, weight_fc='', weight_model='', workers=0)


NameError: ignored

In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
label = []
labelList = []
file = open('C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTrain.txt', 'r')
for line in file.readlines():
  list_new = line.strip().split('\t')
  imgPath, imgOccPath, label_new = list_new[1], list_new[2], list_new[0]
  label_new = label_new[:-14]
  labelList.append(label_new)

print(len(labelList))
# label_2 = label_encoder.fit_transform(labelList)
# label_2 = torch.Tensor(label_2)
# label_2 = label_2.type(torch.int64)
# label_2 = torch.unsqueeze(label_2, 0)

263


In [ ]:
from __future__ import print_function
from __future__ import division
import argparse
import os
import time
import numpy as np
from PIL import Image, ImageFile

import torch
import torch.utils.data
import torch.optim
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

cudnn.benchmark = True
# Training settings
parser = argparse.ArgumentParser(description='PyTorch CosFace')

parser.add_argument('--is_gray', type=bool, default=False,
                    help='Transform input image to gray or not  (default: False)')
parser.add_argument('--gpus', type=str, default='0,1,2,3')
parser.add_argument('--no_cuda', type=bool, default=False,
                    help='disables CUDA training')
parser.add_argument('--classifier_type', type=str, default='MCP')
parser.add_argument('-f', required=False)
args, unknown = parser.parse_known_args()
model_test = LResNet50E_IR_Sia(is_gray=args.is_gray)
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpus
args.cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device('cpu')
pretraining = torch.load('C:/Jun25-23-39-23/CosFace_10_checkpoint.pth')
model_test.load_state_dict(pretraining['model_state_dict'], strict=False)
classifier = {
  'MCP': MarginCosineProduct(512, 200),
  'AL' : AngleLinear(512, 200),
  'L'  : torch.nn.Linear(512, 200, bias=False)
}[args.classifier_type]
classifier.load_state_dict(torch.load('C:/Jun25-23-39-23/CosFace_10_checkpoint_classifier.pth'))
classifier = classifier.to(device)
label_2 = label_2.to(device)


model_test = model_test.to(device) # Set model to gpu

image = open('C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled/Alejandro_Toledo/Alejandro_Toledo_0030.jpg', 'rb')
image_occ = open('C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled/lfw_train/Alejandro_Toledo/Alejandro_Toledo_0030.jpg', 'rb')

if args.is_gray:
   train_transform = transforms.Compose([
      transforms.Grayscale(),
      transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
      transforms.Normalize(mean=(0.5,), std=(0.5,))
    ])  # gray
else:
   train_transform = transforms.Compose([
      transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
      transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
   ])


input_image = Image.open(image).convert('RGB') # Dtype is fp32
input_image = torchvision.transforms.Resize((128, 128))(input_image)
input_image_occ = Image.open(image_occ).convert('RGB') # Dtype is fp32
input_image = train_transform(input_image)
input_image_occ = train_transform(input_image_occ)
input_image = input_image.unsqueeze(0)
input_image_occ = input_image_occ.unsqueeze(0)
mask_file = np.loadtxt('C:/Users/Admin/Downloads/mean_mask.txt', dtype='float32')
mask = mask_file.reshape(512, 16, 16)
# Run forward pass
model_test.eval()
with torch.no_grad():
  f_masked, focc_masked, output, output_occ, f_diff, out = model_test(input_image, input_image_occ)
  # m = nn.Softmax(dim=1)
  output = classifier(output, label_2)
  # output = m(output)
  probs = torch.nn.functional.softmax(output, dim=1)
  conf, classes = torch.max(probs, 1)
  conf
  print(output.argmax())
  # label_unique = label_unique.numpy()
  name = label_2[output.argmax()]
  print(labelList[name])
  # print(label_unique[output.argmax()])
  # print(conf)

tensor(15)
Alejandro_Toledo


In [ ]:
print(label_unique)
print(label_2)
print(labelList[5])

tensor([ 44,   3,   4,   5,   6,   7,   8,   9,  11,  13,  14,  15,  16,  17,
         19,  20,  21,  22,  23,  24,  25,  81, 107,  36,  60,  47,  69, 114,
        120, 141,  51, 104, 167, 179,  28,  73, 140, 171, 177,  35,  80, 106,
        143, 184, 199,  50,  54,  62,  66,  75,  77, 154, 191,  88, 105, 139,
        185,   1,  32,  42,  65, 133, 146, 190, 198,  57,  68,  91,  97, 102,
        122, 123, 174, 176, 196, 117, 126, 128, 148, 178,  83,  85,  89, 147,
        162, 183, 186, 187, 197,   2,  56,  92, 135, 152, 158, 163, 192,  43,
         76,  87,  93, 136, 144, 145, 155, 168, 194,   0,  30,  38,  94, 115,
        138, 149, 157, 159, 161, 173,  18,  45,  82, 108, 109, 131, 132, 142,
        153, 156, 160, 165,  12,  31,  48,  52,  86,  90, 124, 125, 195,  59,
         78,  79,  84, 101, 112, 121, 182,  33,  34,  41,  46,  58,  72,  95,
         96, 103, 113, 116, 119, 127, 150, 151, 169, 181, 189, 193,  10,  49,
         53,  64,  74,  98, 110, 111, 129, 130, 134, 137, 166, 1

In [ ]:
# parser.add_argument('--valid_list', type=str, default='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTest.txt',help='path to validating pair list')
i = 0
label_2_list = label_2.tolist()
valid_transform = transforms.Compose([
    transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # range [0.0, 1.0] -> [-1.0,1.0]
])
val_loader = torch.utils.data.DataLoader(
    ImageList_pair(root='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/lfw-deepfunneled/lfw-deepfunneled', fileList='C:/Users/Admin/Downloads/PDSN-master (1)/PDSN-master/dataset/pairsDevTest_fixed.txt',
                  transform=valid_transform),
    batch_size=1, shuffle=False,
    num_workers=0, pin_memory=False, drop_last=False)
with torch.no_grad():
  for batch_idx, (data, data_occ, _, target) in enumerate(val_loader, 1):
      f_masked, focc_masked, output, output_occ, _, _ = model_test(data, data_occ)
      # pred_0 = np.asarray(pred[0])
      # label = np.asarray(label_2)
      # pred = np.asarray(pred[0])
      # pred = torch.from_numpy(pred)
      pred = output
      output = classifier(output, label_2)
      # m = nn.Softmax(dim=1)
      # output = m(output)
      # classify = classifier(pred, label_2)

      print(label_2_list.index(target), output.argmax(), target)
      if (output.argmax().item() == label_2_list.index(target)):
          i = i + 1
  #         print(i)
  # print(i/263)

0 tensor(3) tensor([[0]], dtype=torch.int32)
1 tensor(4) tensor([[1]], dtype=torch.int32)
2 tensor(5) tensor([[2]], dtype=torch.int32)
3 tensor(6) tensor([[3]], dtype=torch.int32)
4 tensor(7) tensor([[4]], dtype=torch.int32)
5 tensor(8) tensor([[5]], dtype=torch.int32)
6 tensor(9) tensor([[6]], dtype=torch.int32)
7 tensor(11) tensor([[7]], dtype=torch.int32)
8 tensor(13) tensor([[8]], dtype=torch.int32)
9 tensor(14) tensor([[9]], dtype=torch.int32)
10 tensor(15) tensor([[10]], dtype=torch.int32)
11 tensor(16) tensor([[11]], dtype=torch.int32)
12 tensor(17) tensor([[12]], dtype=torch.int32)
13 tensor(19) tensor([[13]], dtype=torch.int32)
14 tensor(20) tensor([[14]], dtype=torch.int32)
15 tensor(21) tensor([[15]], dtype=torch.int32)
16 tensor(22) tensor([[16]], dtype=torch.int32)
17 tensor(23) tensor([[17]], dtype=torch.int32)
18 tensor(24) tensor([[18]], dtype=torch.int32)
19 tensor(25) tensor([[19]], dtype=torch.int32)


In [ ]:
label_unique

tensor([ 44,   3,   4,   5,   6,   7,   8,   9,  11,  13,  14,  15,  16,  17,
         19,  20,  21,  22,  23,  24,  25,  81, 107,  36,  60,  47,  69, 114,
        120, 141,  51, 104, 167, 179,  28,  73, 140, 171, 177,  35,  80, 106,
        143, 184, 199,  50,  54,  62,  66,  75,  77, 154, 191,  88, 105, 139,
        185,   1,  32,  42,  65, 133, 146, 190, 198,  57,  68,  91,  97, 102,
        122, 123, 174, 176, 196, 117, 126, 128, 148, 178,  83,  85,  89, 147,
        162, 183, 186, 187, 197,   2,  56,  92, 135, 152, 158, 163, 192,  43,
         76,  87,  93, 136, 144, 145, 155, 168, 194,   0,  30,  38,  94, 115,
        138, 149, 157, 159, 161, 173,  18,  45,  82, 108, 109, 131, 132, 142,
        153, 156, 160, 165,  12,  31,  48,  52,  86,  90, 124, 125, 195,  59,
         78,  79,  84, 101, 112, 121, 182,  33,  34,  41,  46,  58,  72,  95,
         96, 103, 113, 116, 119, 127, 150, 151, 169, 181, 189, 193,  10,  49,
         53,  64,  74,  98, 110, 111, 129, 130, 134, 137, 166, 1

In [ ]:
labelList.index('John_Herrington')

ValueError: ignored

In [ ]:
labelList[55]

'Martina_Hingis'

In [ ]:
classify.argmax()

tensor(0)

In [ ]:
print(label[11])

Adam_Sandler


In [ ]:
print(classifier.weight.shape)

torch.Size([100, 512])


In [ ]:
print(probs[:, 11])

tensor([6.0452e-08])


In [ ]:
probs = torch.nn.functional.softmax(output, dim=1)
conf, classes = torch.max(probs, 1)
conf

tensor([0.0478])

In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"huyleduc2212","key":"e86ef920a2d33beeb365ff1292c938b2"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download jessicali9530/lfw-dataset

 98% 110M/112M [00:05<00:00, 24.9MB/s]
100% 112M/112M [00:05<00:00, 20.7MB/s]


In [ ]:
! kaggle datasets download ntl0601/casia-webface

100% 2.53G/2.53G [02:01<00:00, 28.1MB/s]
100% 2.53G/2.53G [02:01<00:00, 22.3MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
